# Chapter 9. Using DuckDB in the cloud with MotherDuck

## Getting Started with MotherDuck

### Adding Tables

In [ ]:
CREATE OR REPLACE TABLE my_db.main.airlines AS SELECT * FROM read_csv_auto(['airlines.csv']);
-- If you don't see the newly created table, refresh the page

In [ ]:
DROP TABLE IF EXISTS my_db.main.airlines;

### Creating Schemas

In [ ]:
CREATE OR REPLACE SCHEMA my_db.Titanic;

In [ ]:
CREATE OR REPLACE TABLE my_db.Titanic.Titanic_train AS SELECT * FROM read_csv_auto(['titanic.csv']);
-- If you don't see the newly created table, refresh the page

In [ ]:
-- delete the schema, along with databases contained within it
DROP SCHEMA IF EXISTS my_db.Titanic CASCADE;


### Sharing Databases

In [ ]:
-- Run this snippet to attach the database
ATTACH './md:_share/my_db_GGG/74c9d8c0-d8ed-4fbf-a7b7-2e728ac150af';

In [ ]:
UPDATE SHARE my_db_WeiMengLee;

In [ ]:
LIST SHARES;

### Creating a Database

In [ ]:
CREATE DATABASE flights_db;

In [ ]:
USE flights_db;

### Detaching a Database

In [ ]:
DETACH flights_db;

### Using the Databases in MotherDuck

#### Querying Your Database

In [ ]:
SELECT
    Survived, PClass, Sex, Age
FROM my_db.Titanic.titanic_train
WHERE age NOT NULL AND embarked NOT NULL;

### Writing SQL using AI
| Function/pragma | What it does |
|-----------------|--------------------------------------|
|prompt_sql()|A function to generate a SQL statement for your query|
|prompt_query|A pragma to answer questions about your data|
|prompt_fixup()|A function to correct and fix your SQL query|
|prompt_fix_line()|A function to correct and fix your SQL query line-by-line|
|prompt_schema()|A function to help you understand the contents of a database|
|prompt_explain()|A function to help you understand a SQL query|

In [ ]:
USE my_db;
CALL prompt_sql("How many people who were alone survived");

In [ ]:
SELECT COUNT(*) FROM Titanic.titanic_train WHERE SibSp = 0 AND Parch = 0 AND Survived = 1;

In [ ]:
CALL prompt_sql("Calculate the survival chance for each age group")

In [ ]:
SELECT
    CASE
        WHEN Age < 10 THEN '0-9'
        WHEN Age BETWEEN 10 and 19 THEN '10-19'
        WHEN Age BETWEEN 20 and 29 THEN '20-29'
        WHEN Age BETWEEN 30 and 39 THEN '30-39'
        WHEN Age BETWEEN 40 and 49 THEN '40-49'
        WHEN Age BETWEEN 50 and 59 THEN '50-59'
        WHEN Age BETWEEN 60 and 69 THEN '60-69'
        WHEN Age >= 70 THEN '70+'
        ELSE 'Unknown'
    END AS age_group,
    AVG(Survived) * 100 AS survival_chance
FROM Titanic.Titanic_train
GROUP BY age_group
ORDER BY age_group;

In [ ]:
PRAGMA prompt_query('How many airlines are there?')

In [ ]:
CALL prompt_explain("""
SELECT
    CASE
        WHEN Age < 10 THEN '0-9'
        WHEN Age BETWEEN 10 and 19 THEN '10-19'
        WHEN Age BETWEEN 20 and 29 THEN '20-29'
        WHEN Age BETWEEN 30 and 39 THEN '30-39'
        WHEN Age BETWEEN 40 and 49 THEN '40-49'
        WHEN Age BETWEEN 50 and 59 THEN '50-59'
        WHEN Age BETWEEN 60 and 69 THEN '60-69'
        WHEN Age >= 70 THEN '70+'
        ELSE 'Unknown'
    END AS age_group,
    AVG(Survived) * 100 AS survival_chance
FROM Titanic.Titanic_train
GROUP BY age_group
ORDER BY age_group;
""")

## Using MotherDuck Through the DuckDB CLI

### Connecting to MotherDuck

In [ ]:
duckdb

In [ ]:
ATTACH 'md:';

In [ ]:
SHOW databases;

In [ ]:
SELECT current_database();

In [ ]:
SELECT current_schema();

In [ ]:
SELECT
    Survived, PClass, Sex, Age
FROM my_db.Titanic.titanic_train
WHERE age NOT NULL AND embarked NOT NULL;

In [ ]:
USE my_db.Titanic;

In [ ]:
SELECT current_database();

In [ ]:
SELECT current_schema();

In [ ]:
SELECT count(*) FROM titanic_train;

### Creating Databases on MotherDuck

In [ ]:
CREATE OR REPLACE TABLE my_db.main.airports AS SELECT * FROM read_csv_auto(['./datasets/flights/airports.csv']);

In [ ]:
CREATE DATABASE new_db;
use new_db;
CREATE SCHEMA new_schema;
USE new_schema;
CREATE TABLE new_schema.example_table (id INTEGER, name VARCHAR);
INSERT INTO new_schema.example_table VALUES (1, 'Alice'), (2, 'Bob');
SELECT * FROM new_schema.example_table;

### Performing Hybrid Queries

In [ ]:
SELECT * from my_db.main.airlines;

In [ ]:
SELECT AIRLINE, FLIGHT_NUMBER FROM './datasets/flights/flights.csv' LIMIT 10;

In [ ]:
SELECT f.AIRLINE, f.FLIGHT_NUMBER, a.AIRLINE
    FROM (SELECT AIRLINE, FLIGHT_NUMBER FROM './datasets/flights/flights.csv' LIMIT 10) AS f
    JOIN my_db.main.airlines a ON f.AIRLINE = a.IATA_CODE;